In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('333singles.csv', header=None, names=['id','country','competition','single','date'])
data

,id,country,competition,single,date
0,1982BORS01,Serbia,WC1982,3002,1982-06-05
1,1982BRIN01,Germany,WC1982,3059,1982-06-05
2,1982CHIL01,United Kingdom,WC1982,2595,1982-06-05
3,1982FRID01,Czech Republic,WC1982,2911,1982-06-05
4,1982FRID01,USA,WC2003,1712,2003-08-23
...,...,...,...,...,...
459556,2021ZUOS01,China,HangzhouOpenAM2021,3145,2021-04-18
459557,2021ZURC01,Switzerland,OfflineSchwyzSunday2021,1774,2021-08-15
459558,2021ZVER01,Germany,RheinNeckarOpenA2021,1691,2021-10-16
459559,2021ZYGA01,Lithuania,VilniusOpen2021,2107,2021-08-07


In MySQL I have queried this list of 3x3 (Rubik's Cube) results.
- 'id' is a competitor's WCA ID, a distinct code assigned for each competitor at their first competition
- 'country' is the person's nationality
- 'competition' is the name of the WCA competition in which the result took place and 'date' is the first day of that competition
- 'single' is the person's fastest 3x3 solve from this competition (in centiseconds)

In [3]:
array = data.to_numpy()
pbs_list = []
currentId = '1982BORS01'
currentTime = 999999
for row in array:
    if row[3] > 0:
        if row[0] == currentId:
            if row[3] <= currentTime:
                currentTime = row[3]
                pbs_list.append(row)
        else:
            currentId = row[0]
            currentTime = row[3]
            pbs_list.append(row)

pbs = pd.DataFrame(pbs_list).iloc[:,[0,3,4]]
pbs.rename(columns={0:'id',3:'single',4:'date'}, inplace=True)
pbs

,id,single,date
0,1982BORS01,3002,1982-06-05
1,1982BRIN01,3059,1982-06-05
2,1982CHIL01,2595,1982-06-05
3,1982FRID01,2911,1982-06-05
4,1982FRID01,1712,2003-08-23
...,...,...,...
296830,2021ZUOS01,3145,2021-04-18
296831,2021ZURC01,1774,2021-08-15
296832,2021ZVER01,1691,2021-10-16
296833,2021ZYGA01,2107,2021-08-07


In the end I want a date-by-date ranking list which includes each person's best result so far. That's why it made sense to trim the original dataframe down to only PB (personal best) results. To 'pbs', I've only picked the results where the competitor didn't have a faster result previously.
(Note: Competitors sometimes break their PB multiple times in a single competition, and I'm only using the fastest time from each competition. That's why this list is not complete, though sufficient for my purposes.)

In [4]:
dates = pbs['date'].drop_duplicates().sort_values().tolist()
len(dates)

1757

There are results from 1757 different dates. I'll compute rankings from each of these. Keys in 'rankings' will be the distinct dates, while the values will be lists of person-result pairs (where result is the person's PB result).

In [5]:
rankings = {}
for i in range(0,len(dates)):
    if i > 0: # -1 denotes a 'DNF' result, and I don't want these in the rankings
        rankings[dates[i]] = pd.DataFrame(rankings[dates[i-1]] + pbs[pbs['date']==dates[i]].loc[:,['id','single']].to_numpy().tolist()).groupby(0, as_index=False).min().to_numpy().tolist()
    else:
        rankings[dates[i]] = pbs[pbs['date']==dates[i]].loc[:,['id','single']].to_numpy().tolist()
        

In [6]:
# An example showing the rankings for 28th December 2021 (which is the latest date found in the data)
pd.DataFrame(rankings['2021-12-28']).sort_values(1)

,0,1
37479,2015DUYU01,347
85571,2017XURU04,406
54086,2016KOLA02,411
7650,2009ZEMD01,416
18604,2012PONC02,424
...,...,...
141718,2020SOLI02,48631
79317,2017POLK02,49391
7204,2009URBA02,55381
10171,2010PETR02,59353


For each date, I want to calculate the quartiles from the rankings. These will include:
- World Record: The fastest time
- Top X% (where X is 25, 50 or 75): The time needed to be amongst the top X% competitors in 3x3 single rankings

Keys are again dates, while values will be lists of the aforementioned quartiles

In [7]:
quartile_rankings = {}
for date in dates:
    rankings_length = len(rankings[date])
    quartile_rankings[date] = pd.DataFrame(rankings[date]).sort_values(by=1).iloc[[0,int(0.25*rankings_length),int(0.5*rankings_length),int(0.75*rankings_length)],1].to_numpy().tolist()

quartile_rankings

{'1982-06-05': [2295, 2491, 2911, 3117],
 '2003-08-23': [1653, 2432, 3279, 5429],
 '2003-10-11': [1653, 2396, 3393, 5293],
 '2004-01-24': [1476, 2506, 3734, 6294],
 '2004-04-03': [1211, 2595, 3755, 5939],
 '2004-04-16': [1211, 2601, 3683, 5876],
 '2004-04-24': [1211, 2594, 3458, 5855],
 '2004-07-10': [1211, 2479, 3303, 5782],
 '2004-08-07': [1211, 2487, 3431, 5640],
 '2004-10-10': [1211, 2479, 3431, 5785],
 '2004-10-16': [1211, 2449, 3413, 5535],
 '2004-10-23': [1211, 2434, 3398, 5467],
 '2004-11-13': [1211, 2481, 3458, 5640],
 '2004-11-27': [1211, 2479, 3404, 5535],
 '2004-12-11': [1211, 2491, 3458, 5782],
 '2005-01-15': [1211, 2487, 3623, 5867],
 '2005-01-29': [1211, 2481, 3602, 5867],
 '2005-03-27': [1211, 2479, 3623, 5838],
 '2005-04-16': [1211, 2471, 3633, 6011],
 '2005-04-22': [1211, 2426, 3623, 5907],
 '2005-04-23': [1211, 2412, 3602, 5951],
 '2005-05-28': [1211, 2412, 3553, 5867],
 '2005-06-05': [1211, 2412, 3553, 5907],
 '2005-07-10': [1211, 2358, 3482, 5867],
 '2005-08-13': [

I will convert this into a dataframe, and export it to be used in Power BI

In [8]:
quartile_rankings_progression = pd.DataFrame(quartile_rankings.values())
quartile_rankings_progression.rename(columns={0:'World Record',1:'Top 25%',2:'Top 50%',3:'Top 75%'}, inplace=True)
quartile_rankings_progression.insert(loc=0, column='Date', value=quartile_rankings.keys())
quartile_rankings_progression

,Date,World Record,Top 25%,Top 50%,Top 75%
0,1982-06-05,2295,2491,2911,3117
1,2003-08-23,1653,2432,3279,5429
2,2003-10-11,1653,2396,3393,5293
3,2004-01-24,1476,2506,3734,6294
4,2004-04-03,1211,2595,3755,5939
...,...,...,...,...,...
1752,2021-12-12,347,1599,2491,4060
1753,2021-12-18,347,1599,2491,4059
1754,2021-12-19,347,1599,2490,4059
1755,2021-12-26,347,1599,2490,4059


In [9]:
quartile_rankings_progression.to_csv('333single_quantiles_progression.csv', index=False)